In [11]:
# Importo le librerie necessarie per il codice

import numpy as np
import scipy as sc
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
pd.set_option('display.max_columns', None)

# Elenco dei file da leggere
files = ['attaccanti_23.xlsx', 'attaccanti_22.xlsx', 'attaccanti_21.xlsx', 'attaccanti_20.xlsx', 'attaccanti_19.xlsx']

# Faccio un dizionario per mappare i file agli anni
year_mapping = {'attaccanti_23.xlsx': 2023,'attaccanti_22.xlsx': 2022,'attaccanti_21.xlsx': 2021,'attaccanti_20.xlsx': 2020,'attaccanti_19.xlsx': 2019}


Iniziamo con un'analisi preliminare, escludendo le colonne relative al nome, la nazionalità e la squadra. Non verranno mai considerate in questa sezione, in quanto inutili per questo tipo di analisi 


In [12]:
# Questa parte di codice crea 5 figure che rappresentano il riquadro di correlazione fra le statistiche
# suddiviso per annata
correlations = []
years = []

for file in files:
    # Leggi il file Excel
    data_frame = pd.read_excel(file)
    data_frame['Duels'] = data_frame['Won'] + data_frame['Lost'] # Creo la colonna dei duelli aerei, può essere utile per l'analisi

    # Uso .drop per rimuovere dal data frame quelle statistiche che non voglio usare in questa analisi
    data_frame.drop(columns=['Player', 'Nation', 'Pos', 'Squad'], inplace=True)

    # Inserisco questa riga di codice in modo che in ogni momento si possano visualizzare le colonne usate per l'analisi
    print(f'Le colonne usate per questa analisi sono {list(data_frame.columns)}')    

    # Calcolo la matrice di correlazione
    corr_matrix = data_frame.corr()

    # Ottengo l'anno dal file usando il mapping
    year = year_mapping[file]
    years.append(year)

    # Ottengo in output il grafico di correlazione
    fig = px.imshow(corr_matrix, title=f"Matrice di Correlazione - Anno {year}")
    fig.show()


    # Questa parte di codice serve per conoscere, se desiderato, la correlazione di due statistiche specifiche annata per annata
    # Le statistiche di cui si vuole sapere la correlazione vanno riportate al posto di Stat1 e Stat2
    stat1 = 'Gls'
    stat2 = 'Ast'
    if stat1 in corr_matrix.columns and stat2 in corr_matrix.columns:
        correlation_value = corr_matrix.loc[stat1, stat2]
        correlations.append(correlation_value)
        print(f"Anno {year}: Correlazione tra {stat1} e {stat2} = {correlation_value}")

    else:
        correlations.append(None)  # Aggiunge un valore nullo se manca la correlazione


# Infine questa parte realizza un grafico che mostra l'andamento di questo indice nel corso degli anni

if stat1 in corr_matrix.columns and stat2 in corr_matrix.columns:
   correlation_df = pd.DataFrame({'Anno': years,'Correlazione': correlations})
   fig = px.line(correlation_df, x='Anno', y='Correlazione',title=f"Correlazione tra {stat1} e {stat2} nel tempo",markers=True)
   fig.update_xaxes(tickmode='linear', dtick=1)
   fig.show()
else:
    print("Nessun grafico disponibile: una o entrambe le statistiche non sono presenti nel dataset.")

 # Inserisco questa riga di codice in modo che in ogni momento si possano visualizzare le colonne usate per l'analisi
print(f'Le colonne usate per questa analisi sono {list(data_frame.columns)}')
    

Le colonne usate per questa analisi sono ['Age', 'MP', 'Starts', 'Min', 'Gls', 'Ast', 'G+A', 'G-PK', 'PK', 'PKatt', 'CrdY', 'CrdR', 'xG', 'PrgP', 'PrgR', 'Sh', 'SoT', 'SoT%', 'G/Sh', 'G/SoT', 'Dist', 'FK', 'G-xG', 'Cmp', 'Att', 'Cmp%', 'xA', 'KP', 'PPA', 'CrsPA', 'SCA', 'GCA', 'Tkl', 'TklW', 'Mid 3rd', 'Att 3rd', 'Touches', 'Mid 3rd.1', 'Att 3rd.1', 'Att.1', 'Succ', 'Succ%', 'Carries', 'TotDist', '2CrdY', 'Fld', 'Off', 'Crs', 'Won', 'Lost', 'Won%', 'Duels']


Anno 2023: Correlazione tra Gls e Ast = 0.6800939250456844
Le colonne usate per questa analisi sono ['Age', 'MP', 'Starts', 'Min', 'Gls', 'Ast', 'G+A', 'G-PK', 'PK', 'PKatt', 'CrdY', 'CrdR', 'xG', 'PrgP', 'PrgR', 'Sh', 'SoT', 'SoT%', 'G/Sh', 'G/SoT', 'Dist', 'FK', 'G-xG', 'Cmp', 'Att', 'Cmp%', 'xA', 'KP', 'PPA', 'CrsPA', 'SCA', 'GCA', 'Tkl', 'TklW', 'Mid 3rd', 'Att 3rd', 'Touches', 'Mid 3rd.1', 'Att 3rd.1', 'Att.1', 'Succ', 'Succ%', 'Carries', 'TotDist', '2CrdY', 'Fld', 'Off', 'Crs', 'Won', 'Lost', 'Won%', 'Duels']


Anno 2022: Correlazione tra Gls e Ast = 0.6082915727428799
Le colonne usate per questa analisi sono ['Age', 'MP', 'Starts', 'Min', 'Gls', 'Ast', 'G+A', 'G-PK', 'PK', 'PKatt', 'CrdY', 'CrdR', 'xG', 'PrgP', 'PrgR', 'Sh', 'SoT', 'SoT%', 'G/Sh', 'G/SoT', 'Dist', 'FK', 'G-xG', 'Cmp', 'Att', 'Cmp%', 'xA', 'KP', 'PPA', 'CrsPA', 'SCA', 'GCA', 'Tkl', 'TklW', 'Mid 3rd', 'Att 3rd', 'Touches', 'Mid 3rd.1', 'Att 3rd.1', 'Att.1', 'Succ', 'Succ%', 'Carries', 'TotDist', '2CrdY', 'Fld', 'Off', 'Crs', 'Won', 'Lost', 'Won%', 'Duels']


Anno 2021: Correlazione tra Gls e Ast = 0.5526504212795946
Le colonne usate per questa analisi sono ['Age', 'MP', 'Starts', 'Min', 'Gls', 'Ast', 'G+A', 'G-PK', 'PK', 'PKatt', 'CrdY', 'CrdR', 'xG', 'PrgP', 'PrgR', 'Sh', 'SoT', 'SoT%', 'G/Sh', 'G/SoT', 'Dist', 'FK', 'G-xG', 'Cmp', 'Att', 'Cmp%', 'xA', 'KP', 'PPA', 'CrsPA', 'SCA', 'GCA', 'Tkl', 'TklW', 'Mid 3rd', 'Att 3rd', 'Touches', 'Mid 3rd.1', 'Att 3rd.1', 'Att.1', 'Succ', 'Succ%', 'Carries', 'TotDist', '2CrdY', 'Fld', 'Off', 'Crs', 'Won', 'Lost', 'Won%', 'Duels']


Anno 2020: Correlazione tra Gls e Ast = 0.5999035473537215
Le colonne usate per questa analisi sono ['Age', 'MP', 'Starts', 'Min', 'Gls', 'Ast', 'G+A', 'G-PK', 'PK', 'PKatt', 'CrdY', 'CrdR', 'xG', 'PrgP', 'PrgR', 'Sh', 'SoT', 'SoT%', 'G/Sh', 'G/SoT', 'Dist', 'FK', 'G-xG', 'Cmp', 'Att', 'Cmp%', 'xA', 'KP', 'PPA', 'CrsPA', 'SCA', 'GCA', 'Tkl', 'TklW', 'Mid 3rd', 'Att 3rd', 'Touches', 'Mid 3rd.1', 'Att 3rd.1', 'Att.1', 'Succ', 'Succ%', 'Carries', 'TotDist', '2CrdY', 'Fld', 'Off', 'Crs', 'Won', 'Lost', 'Won%', 'Duels']


Anno 2019: Correlazione tra Gls e Ast = 0.5622417652246772


Le colonne usate per questa analisi sono ['Age', 'MP', 'Starts', 'Min', 'Gls', 'Ast', 'G+A', 'G-PK', 'PK', 'PKatt', 'CrdY', 'CrdR', 'xG', 'PrgP', 'PrgR', 'Sh', 'SoT', 'SoT%', 'G/Sh', 'G/SoT', 'Dist', 'FK', 'G-xG', 'Cmp', 'Att', 'Cmp%', 'xA', 'KP', 'PPA', 'CrsPA', 'SCA', 'GCA', 'Tkl', 'TklW', 'Mid 3rd', 'Att 3rd', 'Touches', 'Mid 3rd.1', 'Att 3rd.1', 'Att.1', 'Succ', 'Succ%', 'Carries', 'TotDist', '2CrdY', 'Fld', 'Off', 'Crs', 'Won', 'Lost', 'Won%', 'Duels']


La barra sulla destra spiega come il colore dei vari quadratini indichi la correlazione delle statistiche. Prima di soffermarci sulle statistiche in particolare facciamo delle discussioni preliminari. Come ci aspettiamo, le statistiche relative al minutaggio complessivo e le partite giocate sono ben correlate con la gran parte delle altre statistiche, in quanto un giocatore che gioca di più molto probabilmente registrerà dati più alti. Notiamo però che l'età, sorprendentemente, non è ben correlata con gran parte delle altre statistiche. Ci suggerisce che l'età del giocatore non influisce sulla prestazione di questo e l'impatto complessivo sulla partita. Togliamola quindi dalle colonne prese in considerazione. Concentriamoci su statistiche precise. 

In [13]:
correlations = []
years = []

for file in files:
    data_frame = pd.read_excel(file)
    data_frame['Duels'] = data_frame['Won'] + data_frame['Lost']

    data_frame.drop(columns=['Player', 'Nation', 'Pos', 'Squad','Age', 'MP', 'Starts', 'Min', 'G+A', 'G-PK', 'PK',
       'PKatt', 'CrdY', 'CrdR', 'SoT%',
       'G/Sh', 'G/SoT', 'Dist', 'FK', 'G-xG', 'Cmp', 'Att', 'Cmp%', 'Tkl', 'TklW', 'Mid 3rd', 'Att 3rd',
       'Mid 3rd.1', 'Att 3rd.1', 'Att.1', 'Succ', 'Succ%',
       'Carries', 'TotDist', '2CrdY', 'Crs',
       'Won%'], inplace=True)
    
    corr_matrix = data_frame.corr()
    
    year = year_mapping[file]
    years.append(year)

    fig = px.imshow(corr_matrix, title=f"Matrice di Correlazione - Anno {year}")
    fig.show()

    stat1 = 'Stat1'
    stat2 = 'Stat2'
    if stat1 in corr_matrix.columns and stat2 in corr_matrix.columns:
        
        correlation_value = corr_matrix.loc[stat1, stat2]
        correlations.append(correlation_value)

        print(f"Anno {year}: Correlazione tra {stat1} e {stat2} = {correlation_value}")

    else:
        correlations.append(None)

if stat1 in corr_matrix.columns and stat2 in corr_matrix.columns:
   correlation_df = pd.DataFrame({'Anno': years,'Correlazione': correlations})
   fig = px.line(correlation_df, x='Anno', y='Correlazione',title=f"Correlazione tra {stat1} e {stat2} nel tempo",markers=True)
   fig.update_xaxes(tickmode='linear', dtick=1)
   fig.show()
else:
    print("Nessun grafico disponibile: una o entrambe le statistiche non sono presenti nel dataset.")

print(f'Le colonne usate per questa analisi sono {list(data_frame.columns)}')

Nessun grafico disponibile: una o entrambe le statistiche non sono presenti nel dataset.
Le colonne usate per questa analisi sono ['Gls', 'Ast', 'xG', 'PrgP', 'PrgR', 'Sh', 'SoT', 'xA', 'KP', 'PPA', 'CrsPA', 'SCA', 'GCA', 'Touches', 'Fld', 'Off', 'Won', 'Lost', 'Duels']


In [14]:
correlations = []
years = []

for file in files:
    data_frame = pd.read_excel(file)
    data_frame['Duels'] = data_frame['Won'] + data_frame['Lost']

    data_frame.drop(columns=['Player', 'Nation', 'Pos', 'Squad','Age', 'MP', 'Starts', 'Min', 'G+A', 'G-PK', 'PK',
       'PKatt', 'Sh', 'SoT', 'SoT%',
       'G/Sh', 'G/SoT', 'Dist', 'FK', 'G-xG', 'Cmp%', 'xA', 'KP',
       'PPA', 'CrsPA', 'SCA', 'GCA', 'Tkl', 'TklW', 'Mid 3rd', 'Att 3rd',
       'Mid 3rd.1', 'Att 3rd.1', 'Succ%',
       'Off', 'Crs', 'Duels', 'Won', 'Lost',
       'Won%'], inplace=True)
    
    corr_matrix = data_frame.corr()

    year = year_mapping[file]
    years.append(year)

    fig = px.imshow(corr_matrix, title=f"Matrice di Correlazione - Anno {year}")
    fig.show()

    stat1 = 'Gls'
    stat2 = 'Touches'
    if stat1 in corr_matrix.columns and stat2 in corr_matrix.columns:
        
        correlation_value = corr_matrix.loc[stat1, stat2]
        correlations.append(correlation_value)

        print(f"Anno {year}: Correlazione tra {stat1} e {stat2} = {correlation_value}")

    else:
        correlations.append(None)


if stat1 in corr_matrix.columns and stat2 in corr_matrix.columns:
   correlation_df = pd.DataFrame({'Anno': years,'Correlazione': correlations})
   fig = px.line(correlation_df, x='Anno', y='Correlazione',title=f"Correlazione tra {stat1} e {stat2} nel tempo",markers=True)
   fig.update_xaxes(tickmode='linear', dtick=1)
   fig.show()
else:
    print("Nessun grafico disponibile: una o entrambe le statistiche non sono presenti nel dataset.")

print(f'Le colonne usate per questa analisi sono {list(data_frame.columns)}')

Anno 2023: Correlazione tra Gls e Touches = 0.7191842991930603


Anno 2022: Correlazione tra Gls e Touches = 0.6409602974518894


Anno 2021: Correlazione tra Gls e Touches = 0.708417456398665


Anno 2020: Correlazione tra Gls e Touches = 0.6916902595854186


Anno 2019: Correlazione tra Gls e Touches = 0.6702142606542649


Le colonne usate per questa analisi sono ['Gls', 'Ast', 'CrdY', 'CrdR', 'xG', 'PrgP', 'PrgR', 'Cmp', 'Att', 'Touches', 'Att.1', 'Succ', 'Carries', 'TotDist', '2CrdY', 'Fld']


In [15]:
correlations = []
years = []

for file in files:
    data_frame = pd.read_excel(file)
    data_frame['Duels'] = data_frame['Won'] + data_frame['Lost']

    data_frame.drop(columns=['Player', 'Nation', 'Pos', 'Squad','Age', 'MP', 'Starts', 'Min', 'Gls', 'Ast', 'G+A', 'G-PK', 'PK',
       'PKatt', 'CrdY', 'CrdR', 'xG', 'PrgP', 'PrgR', 'Sh', 'SoT', 'SoT%',
       'G/Sh', 'G/SoT', 'Dist', 'FK', 'G-xG', 'Cmp', 'Att', 'Cmp%', 'xA', 'KP',
       'PPA', 'CrsPA', 'SCA', 'GCA', 'Tkl', 'TklW', 'Mid 3rd', 'Att 3rd',
       'Touches', 'Mid 3rd.1', 'Att 3rd.1', 'Att.1', 'Succ', 'Succ%',
       'Carries', 'TotDist', '2CrdY', 'Fld', 'Off', 'Crs', 'Duels', 'Won', 'Lost',
       'Won%'], inplace=True)

    corr_matrix = data_frame.corr()

    year = year_mapping[file]
    years.append(year)

    fig = px.imshow(corr_matrix, title=f"Matrice di Correlazione - Anno {year}")
    fig.show()

    stat1 = 'Gls'
    stat2 = 'Touches'
    if stat1 in corr_matrix.columns and stat2 in corr_matrix.columns:
        
        correlation_value = corr_matrix.loc[stat1, stat2]
        correlations.append(correlation_value)

        print(f"Anno {year}: Correlazione tra {stat1} e {stat2} = {correlation_value}")

    else:
        correlations.append(None)


if stat1 in corr_matrix.columns and stat2 in corr_matrix.columns:
   correlation_df = pd.DataFrame({'Anno': years,'Correlazione': correlations})
   fig = px.line(correlation_df, x='Anno', y='Correlazione',title=f"Correlazione tra {stat1} e {stat2} nel tempo",markers=True)
   fig.update_xaxes(tickmode='linear', dtick=1)
   fig.show()
else:
    print("Nessun grafico disponibile: una o entrambe le statistiche non sono presenti nel dataset.")

print(f'Le colonne usate per questa analisi sono {list(data_frame.columns)}')

Nessun grafico disponibile: una o entrambe le statistiche non sono presenti nel dataset.
Le colonne usate per questa analisi sono []
